<a href="https://colab.research.google.com/github/nich-popowich/ECE1512_2022W_ProjectRepo_NicholasPopowich/blob/main/Task1_(Project_B).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Project B: Knowledge Distillation for Building Lightweight Deep Learning Models in Visual Classification Tasks

In [1]:
import tensorflow.compat.v2 as tf
import tensorflow_datasets as tfds
from typing import Union

tf.enable_v2_behavior()

builder = tfds.builder('mnist')
BATCH_SIZE = 256
NUM_EPOCHS = 12
NUM_CLASSES = 10  # 10 total classes.

In [2]:
from tensorflow import keras
from tensorflow.keras import layers
#from keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.optimizers import Adam

# Data loading

In [30]:
# Load train and test splits.
def preprocess(x):
  image = tf.image.convert_image_dtype(x['image'], tf.float32)
  class_labels = tf.one_hot(x['label'], builder.info.features['label'].num_classes)
  return image, class_labels


mnist_train = tfds.load('mnist', split='train', shuffle_files=False).cache()
mnist_train = mnist_train.map(preprocess)
mnist_train = mnist_train.shuffle(builder.info.splits['train'].num_examples)
mnist_train = mnist_train.batch(BATCH_SIZE, drop_remainder=True)

mnist_test = tfds.load('mnist', split='test').cache()
mnist_test = mnist_test.map(preprocess).batch(BATCH_SIZE)

In [6]:
mnist_train

<BatchDataset element_spec=(TensorSpec(shape=(256, 28, 28, 1), dtype=tf.float32, name=None), TensorSpec(shape=(256, 10), dtype=tf.float32, name=None))>

# Model creation

In [31]:


# Build CNN teacher.
#cnn_model = tf.keras.Sequential()

# your code start from here for stpe 2
teacher = keras.Sequential(
    [
        keras.Input(shape=(28, 28, 1)),
        layers.Conv2D(32, (3, 3), strides=(1, 1), padding="same"),
        #layers.Conv2D(256, (3, 3), strides=(2, 2), padding="same"),
        layers.ReLU(),
        layers.MaxPooling2D(pool_size=(2, 2), strides=(1, 1), padding="same"),
        #layers.Conv2D(512, (3, 3), strides=(2, 2), padding="same"),
        layers.Conv2D(64, (3, 3), strides=(1, 1), padding="same"),
        layers.ReLU(),
        layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding="same"),
        layers.Flatten(),
        layers.Dropout(0.5),
        layers.Dense(128, activation="relu"),
        layers.Dropout(0.5),
        #layers.Dense(10),
        #layers.Dense(10, activation="softmax")
        layers.Dense(10)
    ],
    name="teacher",
)


# Build fully connected student.
#fc_model = tf.keras.Sequential()


# your code start from here for step 2

student = keras.Sequential(
    [
        keras.Input(shape=(28, 28, 1)),
        layers.Flatten(),
        layers.Dense(784, activation="relu"),
        layers.Dense(784, activation="relu"),
        #layers.Dense(10),
        #layers.Dense(10, activation="softmax")
        layers.Dense(10)
        # model.add(Activation('relu'))
    ],
    name="student",
)




In [10]:
teacher.compile(loss='categorical_crossentropy', optimizer=Adam(), metrics=['accuracy'])

In [11]:
student.compile(loss='categorical_crossentropy', optimizer=Adam(), metrics=['accuracy'])

In [ ]:
teacher.fit(mnist_train, epochs=5)


In [32]:
student.summary()

Model: "student"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_5 (Flatten)         (None, 784)               0         
                                                                 
 dense_12 (Dense)            (None, 784)               615440    
                                                                 
 dense_13 (Dense)            (None, 784)               615440    
                                                                 
 dense_14 (Dense)            (None, 10)                7850      
                                                                 
Total params: 1,238,730
Trainable params: 1,238,730
Non-trainable params: 0
_________________________________________________________________


In [33]:
teacher.summary()

Model: "teacher"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_4 (Conv2D)           (None, 28, 28, 32)        320       
                                                                 
 re_lu_4 (ReLU)              (None, 28, 28, 32)        0         
                                                                 
 max_pooling2d_4 (MaxPooling  (None, 28, 28, 32)       0         
 2D)                                                             
                                                                 
 conv2d_5 (Conv2D)           (None, 28, 28, 64)        18496     
                                                                 
 re_lu_5 (ReLU)              (None, 28, 28, 64)        0         
                                                                 
 max_pooling2d_5 (MaxPooling  (None, 14, 14, 64)       0         
 2D)                                                       

In [34]:
mnist_train

<BatchDataset element_spec=(TensorSpec(shape=(256, 28, 28, 1), dtype=tf.float32, name=None), TensorSpec(shape=(256, 10), dtype=tf.float32, name=None))>

In [44]:
# Train the teacher model
teacher.compile(loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True), optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5), metrics=["accuracy"])


In [45]:
teacher.fit(mnist_train, epochs=5)

Epoch 1/5
234/234 [==============================] - 212s 896ms/step - loss: 1.7718 - accuracy: 0.5536
Epoch 2/5
234/234 [==============================] - 204s 863ms/step - loss: 1.0945 - accuracy: 0.7035
Epoch 3/5
234/234 [==============================] - 204s 865ms/step - loss: 0.7971 - accuracy: 0.7665
Epoch 4/5
234/234 [==============================] - 205s 868ms/step - loss: 0.6513 - accuracy: 0.8063
Epoch 5/5
234/234 [==============================] - 204s 862ms/step - loss: 0.5698 - accuracy: 0.8288


In [47]:
teacher2 = keras.Sequential(
    [
        keras.Input(shape=(28, 28, 1)),
        layers.Conv2D(32, (3, 3), strides=(1, 1), padding="same"),
        #layers.Conv2D(256, (3, 3), strides=(2, 2), padding="same"),
        layers.ReLU(),
        layers.MaxPooling2D(pool_size=(2, 2), strides=(1, 1), padding="same"),
        #layers.Conv2D(512, (3, 3), strides=(2, 2), padding="same"),
        layers.Conv2D(64, (3, 3), strides=(1, 1), padding="same"),
        layers.ReLU(),
        layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding="same"),
        layers.Flatten(),
        layers.Dropout(0.5),
        layers.Dense(128, activation="relu"),
        layers.Dropout(0.5),
        #layers.Dense(10),
        #layers.Dense(10, activation="softmax")
        layers.Dense(10)
    ],
    name="teacher2",
)

In [54]:
mnist_example, =mnist_train.take(1)
image, label = mnist_example


# Teacher loss function

In [72]:
@tf.function
def compute_teacher_loss(images, labels):
  """Compute class knowledge distillation teacher loss for given images
     and labels.

  Args:
    images: Tensor representing a batch of images.
    labels: Tensor representing a batch of labels.

  Returns:
    Scalar loss Tensor.
  """
  class_logits = teacher2(images, training=True)

  # Compute cross-entropy loss for classes.

  # your code start from here for step 3
  cross_entropy_loss_value =tf.keras.losses.categorical_crossentropy(labels, class_logits, from_logits=True)


  return cross_entropy_loss_value

In [73]:
teacher_wo_hd_loss=compute_teacher_loss(image,label)

In [ ]:
teacher_wo_hd_loss

In [63]:
# Define loass function and optimizer
# loss_func = tf.keras.losses.CategoricalCrossentropy(from_logits=True)
# optimizer = tf.keras.optimizers.Adam(learning_rate=1e-5) # Low since we are fine-tuning


In [64]:
# teacher.compile(loss=compute_teacher_loss(), optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5), metrics=["accuracy"])

# Student loss function

In [75]:


# Hyperparameters for distillation (need to be tuned).
ALPHA = 0.5 # task balance between cross-entropy and distillation loss
DISTILLATION_TEMPERATURE = 4. #temperature hyperparameter

def distillation_loss(teacher_logits: tf.Tensor, student_logits: tf.Tensor,
                      temperature: Union[float, tf.Tensor]):
  """Compute distillation loss.

  This function computes cross entropy between softened logits and softened
  targets. The resulting loss is scaled by the squared temperature so that
  the gradient magnitude remains approximately constant as the temperature is
  changed. For reference, see Hinton et al., 2014, "Distilling the knowledge in
  a neural network."

  Args:
    teacher_logits: A Tensor of logits provided by the teacher.
    student_logits: A Tensor of logits provided by the student, of the same
      shape as `teacher_logits`.
    temperature: Temperature to use for distillation.

  Returns:
    A scalar Tensor containing the distillation loss.
  """
 # your code start from here for step 3
  soft_targets = teacher_logits / temperature

  return tf.reduce_mean(
      tf.nn.softmax_cross_entropy_with_logits(
          soft_targets, student_logits / temperature)) * temperature ** 2

def compute_student_loss(images, labels):
  """Compute class knowledge distillation student loss for given images
     and labels.

  Args:
    images: Tensor representing a batch of images.
    labels: Tensor representing a batch of labels.

  Returns:
    Scalar loss Tensor.
  """
  student_class_logits = student(images, training=True)

  # Compute class distillation loss between student class logits and
  # softened teacher class targets probabilities.

  # your code start from here for step 3

  teacher_class_logits = teacher(images, training=False)
  distillation_loss_value =distillation_loss(teacher_class_logits,student_class_logits,DISTILLATION_TEMPERATURE)

  # Compute cross-entropy loss with hard targets.

  # your code start from here for step 3

  cross_entropy_loss_value = tf.keras.losses.categorical_crossentropy(labels, student_class_logits, from_logits=True)
  
  total_loss =ALPHA*cross_entropy_loss_value + (1-ALPHA)*distillation_loss_value

  return total_loss

# Train and evaluation

In [76]:
@tf.function
def compute_num_correct(model, images, labels):
  """Compute number of correctly classified images in a batch.

  Args:
    model: Instance of tf.keras.Model.
    images: Tensor representing a batch of images.
    labels: Tensor representing a batch of labels.

  Returns:
    Number of correctly classified images.
  """
  class_logits = model(images, training=False)
  return tf.reduce_sum(
      tf.cast(tf.math.equal(tf.argmax(class_logits, -1), tf.argmax(labels, -1)),
              tf.float32)), tf.argmax(class_logits, -1), tf.argmax(labels, -1)

In [86]:
def train_and_evaluate(model, compute_loss_fn):
  """Perform training and evaluation for a given model.

  Args:
    model: Instance of tf.keras.Model.
    compute_loss_fn: A function that computes the training loss given the
      images, and labels.
  """

  # your code start from here for step 4
  optimizer = tf.keras.optimizers.Adam(learning_rate=1e-5)

  for epoch in range(1, NUM_EPOCHS + 1):
    # Run training.
    print('Epoch {}: '.format(epoch), end='')
    for images, labels in mnist_train:
      with tf.GradientTape() as tape:
         # your code start from here for step 4
        logits = model(images, training=True)
        loss_value = compute_loss_fn(labels,logits)

      grads = tape.gradient(loss_value, model.trainable_variables)
      optimizer.apply_gradients(zip(grads, model.trainable_variables))

    # Run evaluation.
    num_correct = 0
    num_total = builder.info.splits['test'].num_examples
    for images, labels in mnist_test:
      # your code start from here for step 4
      num_correct += compute_num_correct(model.images,labels)
    print("Class_accuracy: " + '{:.2f}%'.format(
        num_correct / num_total * 100))


In [ ]:
train_and_evaluate(teacher2,tf.keras.losses.CategoricalCrossentropy(from_logits=True))

Epoch 1: 

# Training models

In [ ]:
# your code start from here for step 5 



# Test accuracy vs. tempreture curve

In [ ]:
# your code start from here for step 6


# Train student from scratch

In [18]:
# Build fully connected student.
fc_model_no_distillation = tf.keras.Sequential()

# your code start from here for step 7





def compute_plain_cross_entropy_loss(images, labels):
  """Compute plain loss for given images and labels.

  For fair comparison and convenience, this function also performs a
  LogSumExp over classes, but does not perform class distillation.

  Args:
    images: Tensor representing a batch of images.
    labels: Tensor representing a batch of labels.

  Returns:
    Scalar loss Tensor.
  """
  # your code start from here for step 7

  student_class_logits = fc_model_no_distillation(images, training=True)
  cross_entropy_loss =  tf.keras.losses.categorical_crossentropy(labels, student_class_logits, from_logits=True)
  
  return cross_entropy_loss


train_and_evaluate(fc_model_no_distillation, compute_plain_cross_entropy_loss)

NameError: ignored

# Comparing the teacher and student model (number of of parameters and FLOPs) 

In [ ]:
# your code start from here for step 8


# XAI method to explain models

In [ ]:
# your code start from here for step 9


# Implementing the state-of-the-art KD algorithm

In [ ]:
# your code start from here for step 13
